In [1]:
import chess, os
import numpy as np
import tensorflow as tf

from model import ResNet
import utils
conf = utils.Config()


from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2060, compute capability 7.5


2022-07-20 19:21:16.887146: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 19:21:16.914239: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 19:21:16.914405: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 19:21:16.915245: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
def gen(path):
    planes = None
    output_array = np.zeros([*conf.BOARD_SHAPE, conf.N_PLANES], dtype=conf.PLANES_DTYPE_NP)
    
    for filename in [path]:
    # for filename in glob.glob(os.path.join(database_path, '*.pgn')): # for whole dataset
        with open(os.path.join(os.getcwd(), filename), 'r') as pgn:
            game = chess.pgn.read_game(pgn)

            while game != None:
                whole_game_moves = game.game().mainline_moves()
                board = chess.Board()
                board_history = [board.fen()[:-6]]
                
                for move in whole_game_moves:
                    # the input is the PREVIOUS board
                    planes = utils.update_planes(planes, board, board_history)
                    # inputs.append(planes)
                    
                    # the output is the move from that position
                    mask = utils.mask_moves([move])[0]
                    output_array[mask[0], mask[1], mask[2]] = 1
                    # outputs.append(output_array)

                    yield (planes, output_array) ### yield before resetting the output

                    output_array[mask[0], mask[1], mask[2]] = 0
                    
                    # then you actually push the move preparing for next turn
                    board.push(move)
                    board_history.append(board.fen()[:-6])
                
                game = chess.pgn.read_game(pgn)

# avg = np.average([info[file, "games"] for file in info.keys()])

In [3]:
train_file_path = "/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-11.pgn"  # 19547 samples
val_file_path = "/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-09.pgn"    # 15587 samples

dataset = tf.data.Dataset.from_generator(gen, args = [train_file_path],
    output_signature=(
    tf.TensorSpec((8,8,119), dtype=tf.dtypes.float16),
    tf.TensorSpec((8,8,73), dtype=tf.dtypes.float16)
))

val_dataset = tf.data.Dataset.from_generator(gen, args = [val_file_path],
    output_signature=(
    tf.TensorSpec((8,8,119), dtype=tf.dtypes.float16),
    tf.TensorSpec((8,8,73), dtype=tf.dtypes.float16)
))

ds = dataset.shuffle(conf.BATCH_DIM*1000) \
    .batch(conf.BATCH_DIM, num_parallel_calls=tf.data.AUTOTUNE) \
    .prefetch(tf.data.AUTOTUNE)

val_ds = val_dataset.shuffle(conf.BATCH_DIM*1000) \
    .batch(conf.BATCH_DIM*2, num_parallel_calls=tf.data.AUTOTUNE) \
    .prefetch(tf.data.AUTOTUNE)

2022-07-20 19:21:18.934079: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-20 19:21:18.935240: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 19:21:18.935505: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 19:21:18.935624: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [4]:
model = ResNet()
model.compile(
    optimizer = "adam",
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
    metrics = ["accuracy"]
)

# model.fit(
#     ds,
#     validation_data = val_ds,
#     epochs = 20,
#     steps_per_epoch = 950,
#     workers = 16,
#     use_multiprocessing = True
# )

In [5]:
sam = 0
for sample in ds.take(1):
    sam, _ = sample
    print(np.shape(sam))

model(tf.stack([tf.zeros([*conf.BOARD_SHAPE, conf.TOTAL_PLANES])]*8, axis = 0))

(8, 8, 8, 119)


2022-07-20 18:43:57.262263: E tensorflow/stream_executor/cuda/cuda_blas.cc:232] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-07-20 18:43:57.262305: E tensorflow/stream_executor/cuda/cuda_blas.cc:234] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2022-07-20 18:43:57.262330: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:838 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


InternalError: Exception encountered when calling layer "conv2d" (type Conv2D).

Attempting to perform BLAS operation using StreamExecutor without BLAS support [Op:Conv2D]

Call arguments received by layer "conv2d" (type Conv2D):
  • inputs=tf.Tensor(shape=(8, 8, 8, 119), dtype=float16)

In [ ]:
print()

tf.Tensor([  8   8   8 119], shape=(4,), dtype=int32)
